In [1]:
import pandas as pd
from datetime import datetime
df = pd.read_csv('research_traffic_multiple_labels.csv')
df.shape

(60000, 12)

In [2]:
print(df.dtypes)
print(df['Label'].value_counts()[1])
print(df['Label'].value_counts()[0])
df.head()

Count of Source IP        int64
Port Count                int64
Pair Count Ratio        float64
Packet Count Diff         int64
Lookup Count Diff         int64
Protocol                  int64
Average Packet Count    float64
Average Byte Count      float64
Packet Std Dev          float64
Byte Std Dev            float64
Duration per Flow       float64
Label                     int64
dtype: object
10000
10000


,Count of Source IP,Port Count,Pair Count Ratio,Packet Count Diff,Lookup Count Diff,Protocol,Average Packet Count,Average Byte Count,Packet Std Dev,Byte Std Dev,Duration per Flow,Label
0,545,545,0.0,2,43,6,0.027523,4.788991,0.163601,28.466647,0.668020,3
1,545,545,0.0,10,0,6,0.009174,1.596330,0.095342,16.589551,0.419081,3
2,52,52,0.0,3,0,6,0.000000,0.000000,0.000000,0.000000,0.018792,3
3,545,545,0.0,13,21,6,0.023853,4.150459,0.355016,61.772825,2.946281,3
4,545,545,0.0,2,131,6,0.027523,4.788991,0.163601,28.466647,2.674461,3


In [3]:
# obj_cols=df.dtypes[df.dtypes == "object"].index.values.tolist()
# print(obj_cols)

# from sklearn.preprocessing import LabelEncoder
# #Encode labels of multiple columns at once

# df[obj_cols] = df[obj_cols].astype(str)
# df[obj_cols] = df[obj_cols].apply(LabelEncoder().fit_transform)
# #
# # Print head
# #
# print(df.dtypes)
# df.head()

In [4]:
import numpy as np
from datetime import datetime
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

y = df.Label
X = df.drop(['Label'],axis=1)

LR = LogisticRegression(solver='liblinear')

# Normalize the dataset using MinMaxScaler
normalizer = MinMaxScaler()
X_normalized = normalizer.fit_transform(X)
X = pd.DataFrame(X_normalized, columns=X.columns)
print("Dataset normalized using MinMaxScaler.")

X_train_cv, X_unseen_test, y_train_cv, y_unseen_test = train_test_split(X, y, test_size=0.2, random_state=42)

kf = KFold(n_splits=10, random_state=42, shuffle=True)

Dataset normalized using MinMaxScaler.


In [5]:
# Initialize lists to store metrics
train_accuracies = []
test_accuracies = []
precisions = []
recalls = []
f1_scores = []
confusion_matrices = []
start = datetime.now()

for train_index, test_index in kf.split(X_train_cv):
    X_train, X_test = X_train_cv.iloc[train_index], X_train_cv.iloc[test_index]
    y_train, y_test = y_train_cv.iloc[train_index], y_train_cv.iloc[test_index]
    
    # Train the dt classifier
    LR.fit(X_train, y_train)
    
    # Make predictions on the test fold
    y_pred_test = LR.predict(X_test)
    y_pred_train = LR.predict(X_train)
    
    # Calculate metrics for the test fold
    train_accuracies.append(accuracy_score(y_train, y_pred_train))
    test_accuracies.append(accuracy_score(y_test, y_pred_test))
    precisions.append(precision_score(y_test, y_pred_test, average='weighted'))
    recalls.append(recall_score(y_test, y_pred_test, average='weighted'))
    f1_scores.append(f1_score(y_test, y_pred_test, average='weighted'))
    confusion_matrices.append(confusion_matrix(y_test, y_pred_test))

# Test the final model on unseen data
y_unseen_pred = LR.predict(X_unseen_test)
unseen_accuracy = accuracy_score(y_unseen_test, y_unseen_pred)
unseen_precision = precision_score(y_unseen_test, y_unseen_pred, average='weighted')
unseen_recall = recall_score(y_unseen_test, y_unseen_pred, average='weighted')
unseen_f1 = f1_score(y_unseen_test, y_unseen_pred, average='weighted')
unseen_conf_matrix = confusion_matrix(y_unseen_test, y_unseen_pred)
end = datetime.now()

In [6]:
# Calculate the average confusion matrix for multi-class classification
import numpy as np

# Determine the number of classes from the confusion matrices
num_classes = confusion_matrices[0].shape[0]

# Initialize an array to store the sum of confusion matrices
confusion_matrix_sum = np.zeros((num_classes, num_classes))

# Sum up the confusion matrices from all folds
for cm in confusion_matrices:
    confusion_matrix_sum += cm

# Compute the average confusion matrix
average_conf_matrix = confusion_matrix_sum / len(confusion_matrices)

# Display the average confusion matrix
print("Average Confusion Matrix:")
max_width = max(len("{:.2f}".format(value)) for row in average_conf_matrix for value in row)

# Print each row with formatted values
for row in average_conf_matrix:
    print(" ".join(f"{value:>{max_width}.2f}" for value in row))

Average Confusion Matrix:
512.70  36.00   8.40  46.00   7.70 188.30
 16.50 746.10   0.00   0.00   4.10  27.70
  4.10   0.10 794.50   0.00   0.80   0.00
 25.50   0.00   0.00 774.10   0.00   0.00
  4.70   0.00   0.00   0.00 798.80   0.00
 14.70   0.00   0.00   0.00   0.30 788.90


In [7]:
# Print the metrics for each fold
for i in range(10):
    print(f"Fold {i+1}:")
    print(f"Training Accuracy: {train_accuracies[i]}")
    print(f"Testing Accuracy: {test_accuracies[i]}")
    print(f"Precision: {precisions[i]}")
    print(f"Recall: {recalls[i]}")
    print(f"F1-Score: {f1_scores[i]}")
    print(f"Confusion Matrix:\n{confusion_matrices[i]}\n")

print(f"Average Training Accuracy: {sum(train_accuracies) / 10}")
print(f"Average Testing Accuracy: {sum(test_accuracies) / 10}")
print(f"Average Precision: {sum(precisions) / 10}")
print(f"Average recall: {sum(recalls) / 10}")
print(f"Average F1 Score: {sum(f1_scores) / 10}")

# Print the performance on the unseen data
print("Performance on Unseen Data:")
print(f"Accuracy: {unseen_accuracy}")
print(f"Precision: {unseen_precision}")
print(f"Recall: {unseen_recall}")
print(f"F1-Score: {unseen_f1}")
print(f"Confusion Matrix:\n{unseen_conf_matrix}")
print("excution time: ", end-start)

Fold 1:
Training Accuracy: 0.9196296296296296
Testing Accuracy: 0.9227083333333334
Precision: 0.9260789809850678
Recall: 0.9227083333333334
F1-Score: 0.9192788437711313
Confusion Matrix:
[[481  43  11  37   6 183]
 [ 17 773   0   0   0  30]
 [  4   0 769   0   0   0]
 [ 19   0   0 775   0   0]
 [  4   0   0   0 803   0]
 [ 17   0   0   0   0 828]]

Fold 2:
Training Accuracy: 0.9206018518518518
Testing Accuracy: 0.9154166666666667
Precision: 0.9214798761975275
Recall: 0.9154166666666667
F1-Score: 0.9117923588084492
Confusion Matrix:
[[518  32   8  57   7 207]
 [ 20 722   0   0   6  29]
 [  6   0 806   0   0   0]
 [ 24   0   0 790   0   0]
 [  5   0   0   0 780   0]
 [  5   0   0   0   0 778]]

Fold 3:
Training Accuracy: 0.9202546296296297
Testing Accuracy: 0.916875
Precision: 0.9205872120328296
Recall: 0.916875
F1-Score: 0.9138396871883261
Confusion Matrix:
[[500  33   7  43  11 196]
 [ 16 801   0   0   1  29]
 [  7   0 766   0   1   0]
 [ 34   0   0 791   0   0]
 [  2   0   0   0 767  

In [8]:
# import pickle
# with open('lr_multi.pkl', 'wb') as file:
#     pickle.dump(LR, file)